In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/dataset.zip'

In [1]:
# import reqired libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np

In [2]:
import glob
from sklearn.model_selection import train_test_split

def load(lst):
  data = []
  for npy_file in lst:
      npy_data = np.load(npy_file)
      data.append(npy_data)
  return data

# Load the data
no_images_list = glob.glob('/content/dataset/train/no/*.npy')
sphere_images_list = glob.glob('/content/dataset/train/sphere/*.npy')
vort_images_list = glob.glob('/content/dataset/train/vort/*.npy')

no_images = load(no_images_list)
sphere_images = load(sphere_images_list)
vort_images = load(vort_images_list)

In [3]:
# Combine the data into a single array
X = np.concatenate((no_images[:3000], sphere_images[:3000], vort_images[:3000]), axis=0)

# Create labels for the data
y = np.concatenate((np.zeros(3000), np.ones(3000), np.ones(3000)*2), axis=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 10

In [4]:
# # validation data
# no_images_v_list = glob.glob('/content/dataset/val/no/*.npy')
# sphere_images_v_list = glob.glob('/content/dataset/val/sphere/*.npy')
# vort_images_v_list = glob.glob('/content/dataset/val/vort/*.npy')

# no_images_v = load(no_images_v_list)
# sphere_images_v = load(sphere_images_v_list)
# vort_images_v = load(vort_images_v_list)

In [5]:
# X_val = np.concatenate((no_images_v[:500], sphere_images_v[:500], vort_images_v[:500]), axis=0)
# y_val = np.concatenate((np.zeros(500), np.ones(500), np.ones(500)*2), axis=0)

In [6]:
# Create the training and testing datasets
train_dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.astype(np.float32)),
    torch.from_numpy(y_train.astype(np.int64))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_test.astype(np.float32)),
    torch.from_numpy(y_test.astype(np.int64))
)

val_dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_test.astype(np.float32)),
    torch.from_numpy(y_test.astype(np.int64))
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Model Architecture: The model architecture is a simple convolutional neural network (CNN) consisting of three convolutional layers, each followed by a ReLU activation and max-pooling operation to reduce the spatial dimensions of the output feature maps. The output of the last convolutional layer is flattened and passed through two fully connected layers, each followed by a ReLU activation function, and a final softmax layer for classification.

In [7]:
#Define model architecture
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(64 * 18 * 18, 128),
    nn.ReLU(),
    nn.Linear(128, 3),
    nn.Softmax(dim=1)
)

Loss Function, Optimizer, and Evaluation Metric: The loss function used is Cross Entropy Loss, which is commonly used for multi-class classification problems. The optimizer used is the Adam optimizer, which is a widely used stochastic gradient descent optimizer with adaptive learning rates. The evaluation metric used is the Area Under the Receiver Operating Characteristic Curve (ROC AUC), which measures the model's ability to distinguish between the three classes.

In [8]:
# Define loss function, optimizer, and evaluation metric
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Training the Model: The model is trained using the training dataset and validated using the validation dataset. The number of epochs used is 10, and the batch size is 32. The model is evaluated after each epoch using the validation set, and the loss and AUC score are printed for each epoch.

In [ ]:
# Train the model

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    running_loss /= len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss:.4f}')

torch.save(model.state_dict(), 'lens_classification_model.pth')

Evaluating the Model: Once the model is trained, it is evaluated on the test dataset, and the AUC score and ROC AUC score are computed. The AUC score is computed using the predicted and actual labels, and the ROC curve is plotted using the predicted scores and actual labels.

In [ ]:
# Evaluate the model on test data
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
predictions = []
actuals = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(labels.cpu().numpy())
test_loss /= len(test_loader.dataset)
test_accuracy = test_correct / test_total
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Calculate ROC curve and AUC score
from sklearn.metrics import roc_curve, roc_auc_score
import numpy as np
predictions = np.array(predictions)
actuals = np.array(actuals)
fpr, tpr, thresholds = roc_curve(actuals, predictions[:, 1], pos_label=1)
auc_score = roc_auc_score(actuals, predictions,multi_class='ovo')
print(f'AUC Score: {auc_score:.4f}')

ROC Curve

In [ ]:
import matplotlib.pyplot as plt
# Plot the ROC curve
plt.plot(fpr, tpr, label='ROC curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()
